### Set the Inference server url (replace with your own address)

In [1]:
#inference_server_url = "http://test-llm10-predictor.test-llm.svc.cluster.local"
inference_server_url = "https://test-llm10-test-llm.apps.cluster-bjzm6.bjzm6.sandbox2625.opentlc.com"

### Install dependencies

In [2]:
!pip install -q langchain==0.1.9 openai==1.13.3


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip -q install pyOpenSSL


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Extract self signed SSL certificate

In [4]:
import os
import socket
import OpenSSL
import socket
from cryptography.hazmat.primitives import serialization
from urllib.parse import urlparse

def save_srv_cert(host, port=443) :
    dst = (host, port)
    sock = socket.create_connection(dst)
    context = OpenSSL.SSL.Context(OpenSSL.SSL.SSLv23_METHOD)
    connection = OpenSSL.SSL.Connection(context, sock)
    connection.set_tlsext_host_name(host.encode('utf-8'))
    connection.set_connect_state()
    try:
        connection.do_handshake()
        certificate = connection.get_peer_certificate()
    except:
        certificate = connection.get_peer_certificate()
    pem_file = certificate.to_cryptography().public_bytes(serialization.Encoding.PEM)
    cert_filename = f"cert-{host}.cer"
    with open(cert_filename, "w") as fout:
        fout.write(pem_file.decode('utf8'))
    return cert_filename

# Extract the hostname"
hostname = urlparse(inference_server_url).netloc
os.environ["SSL_CERT_FILE"] = save_srv_cert(hostname, port=443)

### Preparation for LLM invocation

In [5]:
# Imports
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import VLLMOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

### Create the LLM instance

In [6]:
# LLM definition
llm = VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base= f"{inference_server_url}/v1",
    model_name="/mnt/models/",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

### Create the prompt

In [7]:
template="""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Current conversation:
{history}
Human: {input}
AI:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

### Add some memory for the conversation

In [8]:
memory = ConversationBufferMemory()

### Example using conversation verbose mode. This way we can see the memory buffer

In [9]:
# Verbose mode is intentionally set to True so you can see the prompt and the history from the buffer memory
conversation = ConversationChain(llm=llm,
                                 prompt=PROMPT,
                                 verbose=True,
                                 memory=memory
                                )

In [10]:
first_input = "Describe Paris in 100 words or less."
conversation.predict(input=first_input);



> Entering new ConversationChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Current conversation:

Human: Describe Paris in 100 words or less.
AI:
[/INST]

Paris, the City of Light, is an enchanting metropolis renowned for its iconic landmarks, rich history, and exquisite art scene. Known as the "City of Love," it's home to the Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and Montmartre. Parisian streets are filled with charming cafes, patisseries, and boutiques. Its

In [11]:
second_input = "Is there a river?"
conversation.predict(input=second_input);



> Entering new ConversationChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Current conversation:
Human: Describe Paris in 100 words or less.
AI: Paris, the City of Light, is an enchanting metropolis renowned for its iconic landmarks, rich history, and exquisite art scene. Known as the "City of Love," it's home to the Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and Montmartre. Parisian streets are filled with charming cafes, patisseries, and boutiques. Its vibrant c

### Example using non-verbose mode. The output has only the response without the previous chat items.

In [12]:
conversation2 = ConversationChain(llm=llm,
                                 prompt=PROMPT,
                                 verbose=False,
                                 memory=memory
                                )

In [13]:
# Let's clear the previous conversation first
memory.clear()

In [14]:
first_input = "Can you describe London in 100 words?"
conversation2.predict(input=first_input);

London, the vibrant capital city of the United Kingdom, is a global cultural hub teeming with history, art, and modern innovation. Known for iconic landmarks like the Tower of London, Buckingham Palace, and the British Museum, it's a city rich in heritage. The River Thames winds through its heart, offering scenic views and transportation via the iconic red double-decker buses and the Tube. London's diverse neighborhoods, from Soho's bustling streets to the tranquil Regent's Park, showcase its multicultural population and world-class cuisine. Its energy and diversity make it an unforgettable destination.

In [15]:
# This is a follow-up question without context to showcase the conversation memory
second_input = "Is there a river?"
conversation2.predict(input=second_input);

Yes, the River Thames runs through London. It's an essential part of the city's landscape and history, providing scenic views and transportation via various watercraft.